# Generate Fix Commit Tree

In [28]:
import pandas as pd

In [29]:
df = pd.read_json('../dataset/sstubsLarge')

In [30]:
print('Rows: {}, Columns: {}'.format(*df.shape))

Rows: 63923, Columns: 14


In [31]:
print('**Columns:**')
print(*df.columns, sep='\n')

**Columns:**
bugType
fixCommitSHA1
fixCommitParentSHA1
bugFilePath
fixPatch
projectName
bugLineNum
bugNodeStartChar
bugNodeLength
fixLineNum
fixNodeStartChar
fixNodeLength
sourceBeforeFix
sourceAfterFix


In [32]:
cnt = 0
def is_consecutive_fix(sf):
    global cnt
    cnt += 1
    return sf.fixCommitSHA1.isin(sf.fixCommitParentSHA1).values.sum() > 0

groups = df.groupby(['projectName', 'bugFilePath', 'bugLineNum'])
print(f'{groups.ngroups} groups')
groups = groups.filter(is_consecutive_fix)
print(f'{cnt} groups filtered')

40954 groups
40954 groups filtered


In [33]:
groups.shape

(1076, 14)

In [34]:
import sqlite3
conn = sqlite3.connect('../database/sstubs.db')
cursor = conn.cursor()

In [61]:
query = '''SELECT count(*)
    FROM (SELECT * 
    FROM sstubs_large AS b1
        INNER JOIN sstubs_large AS b2
        ON b1.child = b2.parent AND b1.project = b2.project AND b1.file = b2.file AND b1.line = b2.line 
    GROUP BY b1.child, b1.parent, b1.project, b1.line, b1.file, b1.type)'''
for res in cursor.execute(query):
    print(*res)


399


In [60]:
query = '''SELECT count(*) FROM sstubs_large WHERE (child, project, file, line) IN (
                SELECT parent, project, file, line FROM sstubs_large
           ) '''
for row in cursor.execute(query):
    print(*row)


399
